### 최종

In [10]:
### 0. 임포트
# API 호출 관련
import bs4
import json
import requests
import psycopg2
from os import name
import numpy  as np
import pandas as pd
from tqdm import tqdm

# 날짜 관련
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

### 1. 변수 생성
table = 'm1.land_trade' # Table
month_list = pd.date_range(start='2021.07', end='2021.12', freq='MS').strftime('%Y%m') # 과거
# month_list = [(date.today() - relativedelta(months = 1)).strftime('%Y%m')] # 전월

# 시군구코드 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    response.encoding = 'utf-8'
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:5]) # 법정동 단위
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

### 2. 토지실거래가 API 호출
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcLandTrade'
serviceKey = 'sJuqUegKT1Q0R4IRDqoJklMK8xDxru+rNu+K5JXRh1mbeH4tFTWwQYWVK3xdbbJCCe14u5bQyl43S28/5TtfrA=='

for month in month_list:
    # 리스트 생성
    row_list   = [] # 행 값
    name_list  = [] # 열이름 값
    value_list = [] # 데이터 값
    DF = pd.DataFrame() # 최종 값
    
    # API 요청
    for code in tqdm(lawd_cd_unique):
        params = {
              'serviceKey' : serviceKey
            , 'LAWD_CD'  : code  # 법정동 단위
            , 'DEAL_YMD' : month # 월 단위
        }
        response = requests.get(url, params=params)
        content  = response.text

        # item 태그 분리
        xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
        rows    = xml_obj.findAll('item')

        # 데이터 수집
        for i in range(0, len(rows)):
            columns = rows[i].find_all()

            for j in range(0,len(columns)):
                name_list.append(columns[j].name)
                value_list.append(columns[j].text)

            row_list = value_list
            df_each  = pd.DataFrame([row_list], columns=name_list)
            DF = pd.concat([DF, df_each]).reset_index(drop = True)

            # 초기화
            name_list  = []
            value_list = []
    
    # 전처리
    DF = DF.replace(' ', None)
    DF = DF.replace(np.nan, None)
    if '구분' in DF.columns:
        DF = DF.drop(['구분'], axis = 1)
    
    # rename
    DF = DF.rename(columns = {"거래금액":"deal_amount",
                              "거래면적":"building_area",
                              "거래유형":"trade_type",
                              "년":"deal_year",
                              "법정동":"dong",
                              "시군구":"sigungu",
                              "용도지역":"land_use",
                              "월":"deal_month",
                              "일":"deal_day",
                              "중개사소재지":"broker_location",
                              "지목":"land_category",
                              "지역코드":"regional_code",
                              "해제사유발생일":"unregister_day",
                              "해제여부":"unregister_status"
                              })
    
    # dtype 변경
    DF['deal_amount'] = DF['deal_amount'].apply(lambda x: x.replace(',', '')).astype('int64')
    
    # 날짜 두 자리수로 맞추기
    DF['deal_day'] = ('0' + DF['deal_day'].astype(str)).str[-2:]
    DF['deal_month'] = ('0' + DF['deal_month'].astype(str)).str[-2:]
    
    ### 3. DB INSERT
    # execute_mogrify
    def execute_mogrify(conn, df, table, val):
        # Create a list of tuples from the dataframe values
        tuples = [tuple(x) for x in df.to_numpy()]
        # Comma-separated dataframe columns
        cols = ','.join(list(df.columns))
        # SQL query to execute
        cursor = conn.cursor()
        values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                  tuples]
        query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

        try:
            cursor.execute(query, tuples)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        cursor.close()
    
    # values 설정
    val = '(' + ('%s,' * len(DF.columns))[:-1] + ')'

    # insert
    for j in range(0, len(DF), 10000):

        # DB Connect
        conn = psycopg2.connect(
            host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
            port=5439,
            dbname='dev',
            user='awsuser',
            password='cremaoAdmin1234qwer!!'
        )

        # 10000개 단위로 나누기
        tmp = DF[j:j + 10000]

        # insert
        execute_mogrify(conn, tmp, table, val)
        print(datetime.datetime.now(), ' : ', j)
        tmp = pd.DataFrame()

    # row count
    print(len(DF))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
### 1. 변수 생성
table = 'm1.land_trade' # Table
month_list = pd.date_range(start='2021.07', end='2021.12', freq='MS').strftime('%Y%m') # 과거

### 2. 토지실거래가 API 호출
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcLandTrade'
serviceKey = 'sJuqUegKT1Q0R4IRDqoJklMK8xDxru+rNu+K5JXRh1mbeH4tFTWwQYWVK3xdbbJCCe14u5bQyl43S28/5TtfrA=='

for month in month_list:
    # 리스트 생성
    row_list   = [] # 행 값
    name_list  = [] # 열이름 값
    value_list = [] # 데이터 값
    DF = pd.DataFrame() # 최종 값
    
    # API 요청
    for code in tqdm(lawd_cd_unique):
        params = {
              'serviceKey' : serviceKey
            , 'LAWD_CD'  : code  # 법정동 단위
            , 'DEAL_YMD' : month # 월 단위
        }
        response = requests.get(url, params=params)
        content  = response.text

        # item 태그 분리
        xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
        rows    = xml_obj.findAll('item')

        # 데이터 수집
        for i in range(0, len(rows)):
            columns = rows[i].find_all()

            for j in range(0,len(columns)):
                name_list.append(columns[j].name)
                value_list.append(columns[j].text)

            row_list = value_list
            df_each  = pd.DataFrame([row_list], columns=name_list)
            DF = pd.concat([DF, df_each]).reset_index(drop = True)

            # 초기화
            name_list  = []
            value_list = []
    
    # 전처리
    DF = DF.replace(' ', None)
    DF = DF.replace(np.nan, None)
    if '구분' in DF.columns:
        DF = DF.drop(['구분'], axis = 1)
    
    # rename
    DF = DF.rename(columns = {"거래금액":"deal_amount",
                              "거래면적":"building_area",
                              "거래유형":"trade_type",
                              "년":"deal_year",
                              "법정동":"dong",
                              "시군구":"sigungu",
                              "용도지역":"land_use",
                              "월":"deal_month",
                              "일":"deal_day",
                              "중개사소재지":"broker_location",
                              "지목":"land_category",
                              "지역코드":"regional_code",
                              "해제사유발생일":"unregister_day",
                              "해제여부":"unregister_status"
                              })
    
    # dtype 변경
    DF['deal_amount'] = DF['deal_amount'].apply(lambda x: x.replace(',', '')).astype('int64')
    
    # 날짜 두 자리수로 맞추기
    DF['deal_day'] = ('0' + DF['deal_day'].astype(str)).str[-2:]
    DF['deal_month'] = ('0' + DF['deal_month'].astype(str)).str[-2:]
    
    ### 3. DB INSERT
    # execute_mogrify
    def execute_mogrify(conn, df, table, val):
        # Create a list of tuples from the dataframe values
        tuples = [tuple(x) for x in df.to_numpy()]
        # Comma-separated dataframe columns
        cols = ','.join(list(df.columns))
        # SQL query to execute
        cursor = conn.cursor()
        values = [cursor.mogrify(val, tup).decode('utf8') for tup in
                  tuples]
        query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

        try:
            cursor.execute(query, tuples)
            conn.commit()
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error: %s" % error)
            conn.rollback()
            cursor.close()
            return 1
        cursor.close()
    
    # values 설정
    val = '(' + ('%s,' * len(DF.columns))[:-1] + ')'

    # insert
    for j in range(0, len(DF), 10000):

        # DB Connect
        conn = psycopg2.connect(
            host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
            port=5439,
            dbname='dev',
            user='awsuser',
            password='cremaoAdmin1234qwer!!'
        )

        # 10000개 단위로 나누기
        tmp = DF[j:j + 10000]

        # insert
        execute_mogrify(conn, tmp, table, val)
        print(datetime.datetime.now(), ' : ', j)
        tmp = pd.DataFrame()

    # row count
    print(len(DF))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [34:13<00:00,  8.02s/it]


2022-09-24 20:10:29.371676  :  0
2022-09-24 20:10:31.609493  :  10000
2022-09-24 20:10:34.166850  :  20000
2022-09-24 20:10:37.028610  :  30000
2022-09-24 20:10:39.299014  :  40000
2022-09-24 20:10:42.281836  :  50000
2022-09-24 20:10:43.348788  :  60000
60815


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [29:49<00:00,  6.99s/it]


2022-09-24 20:40:35.984301  :  0
2022-09-24 20:40:39.608862  :  10000
2022-09-24 20:40:44.378970  :  20000
2022-09-24 20:40:47.613401  :  30000
2022-09-24 20:40:50.974725  :  40000
2022-09-24 20:40:52.981440  :  50000
57231


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [22:23<00:00,  5.25s/it]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [31:01<00:00,  7.27s/it]


2022-09-24 21:34:38.492290  :  0
2022-09-24 21:34:41.009358  :  10000
2022-09-24 21:34:44.088900  :  20000
2022-09-24 21:34:46.579525  :  30000
2022-09-24 21:34:51.404870  :  40000
2022-09-24 21:34:54.873814  :  50000
58227


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 256/256 [37:04<00:00,  8.69s/it]


Error: value too long for type character varying(50)

2022-09-24 22:12:01.331444  :  0
Error: value too long for type character varying(50)

2022-09-24 22:12:02.775387  :  10000
2022-09-24 22:12:05.332465  :  20000
2022-09-24 22:12:07.762861  :  30000
2022-09-24 22:12:11.597756  :  40000
2022-09-24 22:12:14.034505  :  50000
Error: value too long for type character varying(50)

2022-09-24 22:12:14.801148  :  60000
61749


 50%|██████████████████████████████████████████████████████▍                                                     | 129/256 [09:10<13:40,  6.46s/it]

### TEST

In [1]:
### 0. 임포트
# API 호출 관련
import bs4
import json
import requests
import psycopg2
from os import name
import numpy  as np
import pandas as pd
from tqdm import tqdm

# 날짜 관련
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

In [4]:
# 기준월
# month_list = pd.date_range(start='2019.01', end='2022.08', freq='MS').strftime('%Y%m') # 과거
month_list = [(date.today() - relativedelta(months = 1)).strftime('%Y%m')] # 전월

In [6]:
# Table
# table = 'm1.rtms_trade'

# 시군구코드 API
lawd_cd_df = pd.DataFrame()
lawd_cd_list = []
lawd_cd_unique = []

for i in range(1, 22):
    
    url = 'http://apis.data.go.kr/1741000/StanReginCd/getStanReginCdList'
    serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
    params = {
        'serviceKey' : serviceKey,
        'pageNo' : str(i),
        'numOfRows' : '1000',
        'type' : 'json'
    }
    response = requests.get(url, params=params)
    content = response.text
    
    # Json List 변환
    lawd_cd_df = pd.DataFrame(pd.json_normalize(data = json.loads(content), record_path = 'StanReginCd')['row'][1])
    lawd_cd_list = list(lawd_cd_df['locatjumin_cd'].str[:5]) # 법정동 단위
    
    # 시군구코드 unique하게 남기기
    for x in lawd_cd_list:
        if x not in lawd_cd_unique:
            lawd_cd_unique.append(x)

In [7]:
spare = lawd_cd_unique

In [8]:
# 토지실거래가 API
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcLandTrade'
serviceKey = 'sJuqUegKT1Q0R4IRDqoJklMK8xDxru+rNu+K5JXRh1mbeH4tFTWwQYWVK3xdbbJCCe14u5bQyl43S28/5TtfrA=='

for month in month_list:
    
    row_list   = [] # 행 값
    name_list  = [] # 열이름 값
    value_list = [] # 데이터 값
    DF = pd.DataFrame() # 최종 값
    
    # API 요청
    for code in tqdm(lawd_cd_unique):
        params = {
              'serviceKey' : serviceKey
            , 'LAWD_CD'  : code  # 법정동 단위
            , 'DEAL_YMD' : month # 월 단위
        }
        response = requests.get(url, params=params)
        content  = response.text

        # item 태그 분리
        xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
        rows    = xml_obj.findAll('item')

        # 데이터 수집
        for i in range(0, len(rows)):
            columns = rows[i].find_all()

            for j in range(0,len(columns)):
                name_list.append(columns[j].name)
                value_list.append(columns[j].text)

            row_list = value_list
            df_each  = pd.DataFrame([row_list], columns=name_list)
            DF = pd.concat([DF, df_each]).reset_index(drop = True)

            # 초기화
            name_list  = []
            value_list = []

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 277/277 [14:46<00:00,  3.20s/it]


In [ ]:
DF_spare = DF

In [19]:
# 전처리
DF = DF.replace(' ', None)
DF = DF.replace(np.nan, None)
if '구분' in DF.columns:
    DF = DF.drop(['구분'], axis = 1)

In [46]:
# 날짜 두 자리수로 맞추기
DF['deal_day'] = ('0' + DF['deal_day'].astype(str)).str[-2:]
DF['deal_month'] = ('0' + DF['deal_month'].astype(str)).str[-2:]

In [48]:
DF

,deal_amount,거래면적,trade_type,deal_year,dong,sigungu,land_use,deal_month,deal_day,broker_location,land_category,regional_code,unregister_day,unregister_status
0,32000,125.7,중개거래,2022,내당동,서구,근린상업지역,08,26,대구 중구,대,27170,,
1,5000,33,직거래,2022,평리동,서구,제2종일반주거지역,08,05,대구 중구,대,27170,,
2,11120,181.71,중개거래,2022,평리동,서구,제2종일반주거지역,08,07,대구 서구,도로,27170,,
3,480,7.83,중개거래,2022,평리동,서구,제2종일반주거지역,08,07,대구 서구,도로,27170,,
4,400,7.83,중개거래,2022,평리동,서구,제2종일반주거지역,08,07,대구 서구,도로,27170,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39406,2000,310,직거래,2022,표선면 토산리,서귀포시,계획관리지역,08,02,제주 서귀포시,도로,50130,22.08.05,O
39407,3216,80,직거래,2022,표선면 토산리,서귀포시,계획관리지역,08,04,제주 서귀포시,유지,50130,22.08.05,O
39408,41040,565,직거래,2022,표선면 토산리,서귀포시,계획관리지역,08,11,제주 서귀포시,대,50130,22.08.05,O
39409,5700,283.67,직거래,2022,표선면 토산리,서귀포시,계획관리지역,08,12,제주 서귀포시,임야,50130,22.08.05,O


In [22]:
# rename
DF = DF.rename(columns = {"거래금액":"deal_amount",
                          "거래면적":"building_area",
                          "거래유형":"trade_type",
                          "년":"deal_year",
                          "법정동":"dong",
                          "시군구":"sigungu",
                          "용도지역":"land_use",
                          "월":"deal_month",
                          "일":"deal_day",
                          "중개사소재지":"broker_location",
                          "지목":"land_category",
                          "지역코드":"regional_code",
                          "해제사유발생일":"unregister_day",
                          "해제여부":"unregister_status"
                          })

# dtype 변경
DF['deal_amount'] = DF['deal_amount'].apply(lambda x: x.replace(',', '')).astype('int64')

In [ ]:
# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(DF.columns))[:-1] + ')'

# insert
for j in range(0, len(DF), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = DF[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(DF))